# 1. Import the necessary libraries for you to be able to query 4square API

In [2]:
import requests
import json
import pandas as pd
from getpass import getpass
import folium
import os
from dotenv import load_dotenv
from pymongo import MongoClient



In [9]:
token = os.getenv("token")

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [3]:
lat = 41.3977461
lon = 2.187674417

In [4]:
def requests_for_foursquare(query, lat, lon, radius=500, limit=10):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat},{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except Exception as e:
        print("Error:", e)
        return None


res = requests_for_foursquare("coffee", lat, lon, radius=500, limit=10)

In [6]:
res

{'message': 'Invalid request token.'}

In [5]:
names = [(res["results"][i]["name"]) for i in range(10)]
location = [(res["results"][i]["location"]["address"]) for i in range(10)]
new_dict = {}
new_dict["name"] = names
new_dict["location"] = location
df_coffe_shop = pd.DataFrame(new_dict)
df_coffe_shop

KeyError: 'results'

# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [ ]:
lat = 41.3977461
lon = 2.187674417

In [51]:
def requests_for_foursquare(query, lat, lon, limit=5):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat},{lon}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except Exception as e:
        print("Error:", e)
        return None


coffee_shop = requests_for_foursquare("coffee", lat, lon, limit=5)

In [67]:
names = [(res["results"][i]["name"]) for i in range(5)]
location = [(res["results"][i]["location"]["address"]) for i in range(5)]
lat = [(res["results"][i]["geocodes"]["main"]["latitude"]) for i in range (5)]
lon = [(res["results"][i]["geocodes"]["main"]["longitude"]) for i in range (5)]
new_dict = {}
new_dict["name"] = names
new_dict["location"] = location
new_dict["lat"] = lat
new_dict["lon"] = lon

coffee_shops = pd.DataFrame(new_dict)
coffee_shops

,name,location,lat,lon
0,SKYE Coffee Co,"Calle Pamplona, 88",41.396937,2.191519
1,Three Marks Coffee,"Ausiàs Marc, 151",41.397168,2.183123
2,El Cafè Blau,"Casp, 192",41.398257,2.182922
3,El Bar,"Ribes, 46",41.398897,2.183774
4,Eva-2,"Bolivia, 21",41.401636,2.189006


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [10]:
def foursquare_places(venue, token):
    endpoint = f'https://api.foursquare.com/v3/places/search?query={venue}'

    headers = {
        "accept": "application/json",
        "Authorization": token
    }

    try:
        response = requests.get(endpoint, headers=headers)
        response.raise_for_status()  # Check for any errors in the response
        data = response.json()
        venues = data['results']
        results = []
        for venue in venues:
            name = venue['name']
            location = venue['geocodes']['main']
            lat = location.get('latitude')
            lng = location.get('longitude')
            results.append({
                'name': name,
                'lat': lat,
                'lng': lng
            })
        return results
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return []
    
def export_to_json(results, output_directory, output_file):
    try:
        full_file_path = os.path.join(output_directory, output_file)
        os.makedirs(output_directory, exist_ok=True)
        with open(full_file_path, 'w') as json_file:
            json.dump(results, json_file, indent=4)
        print(f"Data exported to '{full_file_path}'")
        return True
    except Exception as e:
        print(f"Error: {e}")
        return False

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [11]:
results = foursquare_places("caffe", token)
output_directory = 'data/'
output_file_name = 'coffe_shops.json'

result = export_to_json(results, output_directory, output_file_name)


Data exported to 'data/coffe_shops.json'


# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [18]:
client = MongoClient("mongodb://localhost:27017")

db = client["Ironhack"]

data_file = 'data/coffe_shops.json'

collection_name = "ironcoffee"

with open(data_file, 'r') as json_file:
    data = json.load(json_file)

collection = db[collection_name]
collection.insert_many(data)

client.close()
